# Remove host reads from virus bus file using flag

Align to host with -n column in bus file:

In [1]:
import glob

In [2]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
# sample_batch_file = f"{fastq_folder}/batch.txt"

In [3]:
# host_fasta = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"
# host_gtf = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.109.gtf.gz"

# canine_fasta = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz"
# canine_gtf = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.109.gtf.gz"

# canine_macaque_fasta = "/home/laura/projects/virus-watch-data/canine_macaque_ref/combined_genomes.dna.toplevel.fa.gz"

index = "/home/laura/projects/virus-watch-data/canine_macaque_ref/index.idx"
t2g = "/home/laura/projects/virus-watch-data/canine_macaque_ref/t2g.txt"
fa = "/home/laura/projects/virus-watch-data/canine_macaque_ref/transcriptome.fa"

kallisto = "/home/laura/temp/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

In [4]:
host_out_folder = f"{fastq_folder}/../host_flag/host"

In [ ]:
%%time
!/usr/bin/time -v $kallisto bus \
        -n \
        -i $index \
        -o $host_out_folder \
        -t 40 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0

Align to virus with -n column in bus file (without masking):

In [5]:
out_folder = f"{fastq_folder}/../host_flag/virus"

In [6]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_no-host-mask.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"

In [ ]:
%%time
!/usr/bin/time -v $kallisto bus \
        -n \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t 40 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0

### Split virus matrix into reads seen in host and those not seen in host
This has to be done for each batch (SRR library) separately, since the read numbers reset for each batch:

In [17]:
from tqdm import tqdm

In [9]:
# Get list of sample barcodes
sb_file = f"{host_out_folder}/matrix.sample.barcodes"

with open(sb_file, "r") as sb:
    sample_barcodes = sb.read().splitlines()
    
sample_barcodes[:10]

['AAAAAAAAAAAAAAAA',
 'AAAAAAAAAAAAAAAC',
 'AAAAAAAAAAAAAAAG',
 'AAAAAAAAAAAAAAAT',
 'AAAAAAAAAAAAAACA',
 'AAAAAAAAAAAAAACC',
 'AAAAAAAAAAAAAACG',
 'AAAAAAAAAAAAAACT',
 'AAAAAAAAAAAAAAGA',
 'AAAAAAAAAAAAAAGC']

In [18]:
%%time
sample_barcodes_star = f"{host_out_folder}/matrix.sample.barcodes.tmp"

TQDM_BAR_FORMAT = (
    "{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]"
)

with tqdm(total=len(sample_barcodes), bar_format=TQDM_BAR_FORMAT) as pbar:
    for sb in sample_barcodes:
        with open(sample_barcodes_star, "w") as sbs_file:
            sbs_file.write(sb + "*")

        print(f"Capture reads for batch {sb} for virus and host alignments...")
        # Capture records for this batch from virus and host bus files
        !$bustools capture \
            $out_folder/output.bus \
            --capture $sample_barcodes_star \
            --barcode \
            -o $out_folder/output_sample_tmp.bus

        !$bustools capture \
            $host_out_folder/output.bus \
            --capture $sample_barcodes_star \
            --barcode \
            -o $host_out_folder/output_sample_tmp.bus

        print(f"Split viral alignments into host/masked for batch {sb}...")
        # Capture records from virus reads also mapped to host
        bus_name = f"output_{sb}_host.bus"
        !$bustools capture \
            $out_folder/output_sample_tmp.bus \
            --capture <($bustools text -pf $host_out_folder/output_sample_tmp.bus | cut -d$'\t' -f5)  \
            --flags \
            -o $out_folder/$bus_name

        # Capture records from virus reads that did not map to host
        bus_name = f"output_{sb}_virus.bus"
        !$bustools capture \
            $out_folder/output_sample_tmp.bus \
            --capture <($bustools text -pf $host_out_folder/output_sample_tmp.bus | cut -d$'\t' -f5)  \
            --complement \
            --flags \
            -o $out_folder/$bus_name
        
        pbar.update(n=1)

  0%|                                                                                                                           | 0/106 [elapsed: 00:00 remaining: ?]

Capture reads for batch AAAAAAAAAAAAAAAA for virus and host alignments...
Read in 271350844 BUS records, wrote 2453071 BUS records
Read in 8828381950 BUS records, wrote 90771029 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAAAAAA...
Read in 90771029 BUS records
Read in 2453071 BUS records, wrote 44924 BUS records
Read in 90771029 BUS records
Read in 2453071 BUS records, wrote 2408147 BUS records


  1%|█                                                                                                                   | 1/106 [elapsed: 07:20 remaining: 12:50:24]

Capture reads for batch AAAAAAAAAAAAAAAC for virus and host alignments...
Read in 271350844 BUS records, wrote 666473 BUS records
Read in 8828381950 BUS records, wrote 68474890 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAAAAAC...
Read in 68474890 BUS records
Read in 666473 BUS records, wrote 18498 BUS records
Read in 68474890 BUS records
Read in 666473 BUS records, wrote 647975 BUS records


  2%|██▏                                                                                                                 | 2/106 [elapsed: 13:58 remaining: 11:59:48]

...

Capture reads for batch AAAAAAAAAAAACGGA for virus and host alignments...
Read in 271350844 BUS records, wrote 1099633 BUS records
Read in 8828381950 BUS records, wrote 30617474 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAACGGA...
Read in 30617474 BUS records
Read in 1099633 BUS records, wrote 20022 BUS records
Read in 30617474 BUS records
Read in 1099633 BUS records, wrote 1079611 BUS records


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 105/106 [elapsed: 13:55:44 remaining: 07:35]

Capture reads for batch AAAAAAAAAAAACGGC for virus and host alignments...
Read in 271350844 BUS records, wrote 1482646 BUS records
Read in 8828381950 BUS records, wrote 38488143 BUS records
Split viral alignments into host/masked for batch AAAAAAAAAAAACGGC...
Read in 38488143 BUS records
Read in 1482646 BUS records, wrote 28897 BUS records
Read in 38488143 BUS records
Read in 1482646 BUS records, wrote 1453749 BUS records


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [elapsed: 14:02:39 remaining: 00:00]

CPU times: user 11min 20s, sys: 2min 47s, total: 14min 8s
Wall time: 14h 2min 39s


### Clean up cell barcodes using host cell whitelist generated by bustools

In [19]:
host_whitelist_folder = f"/home/laura/projects/virus-watch-data/{bp}/full_index_stringent_dlist_flank1/host_canine"

In [21]:
import os

In [26]:
%%time
with tqdm(total=len(sample_barcodes), bar_format=TQDM_BAR_FORMAT) as pbar:
    for sb in sample_barcodes:
        for ent in ["host", "virus"]:
            bus_name = f"output_{sb}_{ent}.bus"
            
            # Correct barcodes (no need to sort first)
            # This needs to be done before removing the flag column since adding the sample barcodes to the cell barcodes will confuse bustools correct
            !$bustools correct \
                -w $host_whitelist_folder/bustools_whitelist.txt \
                -o $out_folder/output_temp1.bus \
                $out_folder/$bus_name

            # Remove flag column so bustools count does not get confused
            # -a retains the sample barcodes and adds them to the normal cell barcodes
            !$bustools text \
                -apf $out_folder/output_temp1.bus | cut -d$'\t' -f1,2,3,4 \
                | $bustools fromtext -o $out_folder/output_temp2.bus -

            # Sort and count
            !$bustools sort \
                -m 4G \
                -t 30 \
                -o $out_folder/output_temp3.bus \
                $out_folder/output_temp2.bus

            os.makedirs(f"{out_folder}/{sb}/{ent}", exist_ok=True)
            !$bustools count \
                --genecounts \
                -o $out_folder/$sb/$ent/bustools_count/ \
                -g $virus_t2g \
                -e $out_folder/matrix.ec \
                -t $out_folder/transcripts.txt \
                $out_folder/output_temp3.bus
        
        pbar.update(n=1)

  0%|                                                                                                                           | 0/106 [elapsed: 00:00 remaining: ?]

Found 775516 barcodes in the on-list
Processed 44924 BUS records
In on-list = 27663
Corrected    = 4781
Uncorrected  = 12480
Read in 32444 BUS records
Read in 32444 text records
 all fits in buffer
Read in 32444 BUS records
reading time 0s
sorting time 0.01s
writing time 0s
Found 775516 barcodes in the on-list
Processed 2408147 BUS records
In on-list = 366450
Corrected    = 616539
Uncorrected  = 1425158
Read in 982989 BUS records
Read in 982989 text records
partition time: 0.05s
 all fits in buffer
Read in 982989 BUS records
reading time 0.04s
sorting time 0.4s
writing time 0.02s


  1%|█                                                                                                                      | 1/106 [elapsed: 00:21 remaining: 38:11]

Found 775516 barcodes in the on-list
Processed 18498 BUS records
In on-list = 13032
Corrected    = 1699
Uncorrected  = 3767
Read in 14731 BUS records
Read in 14731 text records
 all fits in buffer
Read in 14731 BUS records
reading time 0s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 647975 BUS records
In on-list = 151626
Corrected    = 144271
Uncorrected  = 352078
Read in 295897 BUS records
Read in 295897 text records
partition time: 0.01s
 all fits in buffer
Read in 295897 BUS records
reading time 0.01s
sorting time 0.41s
writing time 0.02s


  2%|██▏                                                                                                                    | 2/106 [elapsed: 00:38 remaining: 32:51]

...

Found 775516 barcodes in the on-list
Processed 20022 BUS records
In on-list = 9531
Corrected    = 2900
Uncorrected  = 7591
Read in 12431 BUS records
Read in 12431 text records
 all fits in buffer
Read in 12431 BUS records
reading time 0.01s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 1079611 BUS records
In on-list = 188331
Corrected    = 264462
Uncorrected  = 626818
Read in 452793 BUS records
Read in 452793 text records
partition time: 0.01s
 all fits in buffer
Read in 452793 BUS records
reading time 0s
sorting time 0.14s
writing time 0.01s


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 105/106 [elapsed: 38:32 remaining: 00:20]

Found 775516 barcodes in the on-list
Processed 28897 BUS records
In on-list = 16161
Corrected    = 3480
Uncorrected  = 9256
Read in 19641 BUS records
Read in 19641 text records
 all fits in buffer
Read in 19641 BUS records
reading time 0s
sorting time 0s
writing time 0s
Found 775516 barcodes in the on-list
Processed 1453749 BUS records
In on-list = 304618
Corrected    = 335196
Uncorrected  = 813935
Read in 639814 BUS records
Read in 639814 text records
partition time: 0.02s
 all fits in buffer
Read in 639814 BUS records
reading time 0.01s
sorting time 0.15s
writing time 0.01s


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [elapsed: 38:50 remaining: 00:00]

CPU times: user 34.6 s, sys: 12.2 s, total: 46.8 s
Wall time: 38min 50s
